In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
try:
    get_ipython().system('ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432')
except:
    pass


ln: failed to create symbolic link '/tmp/.s.PGSQL.5432': File exists


In [2]:
wdir = '../../datamyneDelivery20170728/02_dataCreated/entitiesToInclude/renamed/'

In [8]:
conn = psycopg2.connect(database="datamyneUSexports", user="ram22")

In [4]:
def create_table(conn, data):
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS temp;")
    cur.execute("CREATE TABLE temp (name varchar, time integer, isin varchar);")
    file_object = open(data, 'r')
    op = "COPY temp FROM STDIN WITH CSV"
    cur.copy_expert(sql=op, file=file_object)
    cur.execute("DELETE FROM temp WHERE isin != 'x' OR isin IS NULL;")
    conn.commit()
    file_object.close()
    cur.close()


In [5]:
dunsSource = pd.read_csv('../../datamyneDelivery20170728/02_dataCreated/listOfCompaniesUsedv2.csv')
dunsSource = dunsSource[dunsSource['hasfile'] == 1]

In [9]:
compList = [x for x in zip(dunsSource['duns_nbr_consignee'], dunsSource['nickname'])]

for compData in compList:
    create_table(conn, wdir + compData[1] +'.csv')
    query = "UPDATE billoflading_all as bill SET duns_nbr_shipper = '" + str(compData[0]) + \
    "' WHERE EXISTS (SELECT 1 FROM temp as names WHERE UPPER(bill.shipper_declared) = UPPER(names.name));"
    #"' WHERE (duns_nbr_consignee IS NULL OR duns_nbr_consignee = '') AND EXISTS (SELECT 1 FROM temp as names WHERE UPPER(bill.consignee_declared) = UPPER(names.name));"
    
    cur = conn.cursor()
    
    cur.execute(query)
    cur.execute("DROP TABLE temp;")
    conn.commit()
    cur.close()
    

In [10]:
conn.close()